    全连接层初始化的w参数过大，导致输入softmax数值非常大(百位级别,最大的数为700多，最小的数为-800多)，这将会导致softmax输出的数组中基本上都为0，只有700多的那个数为1，因此在add_layer函数中初始化w的部分除以100，缩小了w，使得输出结果正常。
    在调试过程中遇到代码出现问题，可以试试在会话中打印数据，观察数据的准确性来发现问题的所在，例如可以把下面打印代码的prediction改为xs观察输入数据的正确型。也可以加.shape观察数据形状
    print(sess.run(prediction, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5}))
    在全连接层中，对于x，样本为数为行，属性数为列。y = xw + b。因为tensorflow对张量的定义为(样本数，长，宽，通道数(rgb))

In [ ]:
import numpy as np
import pandas as pd
import os
import struct
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
N_ATTRIBUTE = 784
N_OUTPUT = 10

In [ ]:
path = os.path.join('/home/secret', 'datasets')
def load_mnist(path, kind):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels.idx1-ubyte'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images.idx3-ubyte'
                               % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)
 
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)
 
    return images, labels
train_images, train_labels = load_mnist(path, 'train')
test_images, test_labels = load_mnist(path, 't10k')

In [ ]:
#对图片顺序进行打乱
shuffle_train_index = np.random.permutation(60000)
x_train_shuffle, y_train_shuffle = train_images[shuffle_train_index], train_labels[shuffle_train_index]
shuffle_test_index = np.random.permutation(10000)
x_test_shuffle, y_test_shuffle = test_images[shuffle_test_index], test_labels[shuffle_test_index]

x_test_shuffle = x_test_shuffle.astype(np.float32) / 255
y_test_shuffle = y_test_shuffle.astype(np.float32)
x_train_shuffle = x_train_shuffle.astype(np.float32) / 255
y_train_shuffle = y_train_shuffle.astype(np.float32)

In [ ]:
'''对label进行onehot编码'''
from sklearn.preprocessing import OneHotEncoder
OneHot = OneHotEncoder(sparse=False, categories='auto')
y_train_enconded = OneHot.fit_transform(y_train_shuffle.reshape(-1, 1))
y_test_enconded = OneHot.fit_transform(y_test_shuffle.reshape(-1, 1))

In [ ]:
#全连接层
#对于x，样本为数为行，属性数为列。
# y = xw + b
def add_layer(inputs, in_size, out_size, activation_function=None):
    weight = tf.Variable(tf.random_normal([in_size, out_size]) / 100)
    biases = tf.Variable(tf.zeros([1, out_size]))
    xw_plus_b = tf.matmul(inputs, weight) + biases
    if activation_function == None:
        return xw_plus_b
    else:
        if activation_function == tf.nn.softmax:
            return activation_function(xw_plus_b, axis=1)
        else:
            return activation_function(xw_plus_b)
    
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 0.5})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 0.5})
    return result

def weight_variable(shape):
    # 截断正态分布
    initial = tf.truncated_normal(shape, stddev=0.1) # 方差越小，数据越集中
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):       
    #ksize为核大小2x2x1
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [ ]:
#placeholder
xs = tf.placeholder(tf.float32, [None, N_ATTRIBUTE])
ys = tf.placeholder(tf.float32, [None, N_OUTPUT])
keep_prob = tf.placeholder(tf.float32) # 随机失活dropout的参数
x_image = tf.reshape(xs, [-1, 28, 28, 1])

In [ ]:
## conv1 layer ## 
W_conv1 = weight_variable([5, 5, 1, 32])# 5x5x1  32个 因为对图片进行了same padding，所有图片的长宽不变
b_conv1 = bias_variable([32])# 一个卷积核一个偏置项
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)# output size 28x28x32
h_pool1 = max_pool_2x2(h_conv1) # output size 14x14x32

## conv2 layer ## 
W_conv2 = weight_variable([5, 5, 32, 64])# 5x5x32  64个 因为对图片进行了same padding，所有图片的长宽不变
b_conv2 = bias_variable([64])# 一个卷积核一个偏置项
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)# output size 14x14x64
h_pool2 = max_pool_2x2(h_conv2) # output size 7x7x64

## fullc1 layer ##
#[n_samples, 7, 7, 64] -> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = add_layer(h_pool2_flat, 7*7*64, 1024, activation_function=tf.nn.relu)
#h_fc1_dropout = tf.nn.dropout(h_fc1, rate=keep_prob)

## fullc2 layer ##
prediction = add_layer(h_fc1, 1024, 10, activation_function=tf.nn.softmax)

In [ ]:
cross_entropy_array = [] # 保存cross_entropy(损失)用以绘制图像
# 先sum在mean时除数为sum之后的个数， 如果只有mean则为所有数的个数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction), reduction_indices=1))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        batch_xs = x_train_shuffle[60*i: 60*(i+1), :]
        batch_ys = y_train_enconded[60*i: 60*(i+1), :]
        #print(sess.run(prediction, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5}))
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
        cross_entropy_array.append(sess.run(cross_entropy, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5}))
        if i % 50 == 0:
            print(compute_accuracy(x_test_shuffle, y_test_enconded))

In [ ]:
'''可视化cross_entropy'''
cross_entropy_array = np.array(cross_entropy_array)
plot_x = np.arange(1000)
plt.figure()
plt.plot(plot_x, cross_entropy_array, 'r-')
plt.show()

In [ ]:
arr1 = (np.array([1.79982986e+02, 6.37160217e2, -8.27999634e+02, -3.87886353e+01,
  -4.91421814e+01, -5.50225830e+01,  7.63858093e+02, -8.44356995e+01,
   8.87969849e+02, 8.05480835e+02]) ).reshape([1, -1])
import torch
arr1 = torch.from_numpy(arr1)
soft = torch.nn.Softmax(dim=1)
print(soft(arr1))